# Multimodal RAG System Tutorial

This notebook extends our basic RAG system to handle multiple data types including PDFs, CSV files, JSON, Markdown, HTML, images, and audio files. We'll demonstrate the advanced capabilities of LlamaIndex's `SimpleDirectoryReader` for multimodal data processing.

## What's New in This Tutorial

Building upon our previous RAG system, we now add:
- **Multimodal Document Loading**: CSV, JSON, Markdown, HTML, Images, Audio
- **Advanced SimpleDirectoryReader Features**: File filtering, metadata extraction, custom processors
- **Cross-Modal Queries**: Search across different data types simultaneously
- **Structured Data Integration**: Combine tabular data with unstructured text
- **Visual Content Processing**: Extract information from images and charts

## Supported File Types (Per LlamaIndex Documentation)

According to the [SimpleDirectoryReader documentation](https://developers.llamaindex.ai/python/framework/module_guides/loading/simpledirectoryreader/), the following formats are automatically supported:

- **.csv** - comma-separated values
- **.docx** - Microsoft Word  
- **.epub** - EPUB ebook format
- **.hwp** - Hangul Word Processor
- **.ipynb** - Jupyter Notebook
- **.jpeg, .jpg** - JPEG image
- **.mbox** - MBOX email archive
- **.md** - Markdown
- **.mp3, .mp4** - audio and video
- **.pdf** - Portable Document Format
- **.png** - Portable Network Graphics
- **.ppt, .pptm, .pptx** - Microsoft PowerPoint


## 1. Environment Setup and Configuration

First, let's set up our environment with hardcoded configurations. We'll use OpenRouter for the LLM and local embeddings for cost-effective processing.


In [ ]:
# !pip install -r "../requirements.txt"

In [5]:
# Environment setup with hardcoded configurations
import os
import time
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import pandas as pd
import json

from dotenv import load_dotenv

# Hardcoded configuration
CONFIG = {
    "llm_model": "gpt-5-mini",
    "embedding_model": "local:BAAI/bge-small-en-v1.5",
    "chunk_size": 1024,
    "chunk_overlap": 100,
    "similarity_top_k": 5,
    "data_path": "../data",
    "vector_db_path": "storage/multimodal_vectordb",
    "index_storage_path": "storage/multimodal_index"
}

def setup_environment():
    """
    Setup environment variables and basic configuration.
    
    Returns:
        bool: Success status
    """
    # Load environment variables from .env file
    load_dotenv()
    
    # Disable tokenizer warning
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    
    # Check for required API key
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        print("⚠️  OPENROUTER_API_KEY not found in environment variables")
        print("Please add your OpenRouter API key to a .env file")
        return False
    
    print("✓ Environment variables loaded successfully")
    print(f"✓ LLM Model: {CONFIG['llm_model']}")
    print(f"✓ Embedding Model: {CONFIG['embedding_model']}")
    return True

# Run the setup
success = setup_environment()
if success:
    print("Environment setup complete!")
else:
    print("Environment setup failed!")


✓ Environment variables loaded successfully
✓ LLM Model: gpt-5-mini
✓ Embedding Model: local:BAAI/bge-small-en-v1.5
Environment setup complete!


## 2. LlamaIndex Configuration for Multimodal Data

Let's configure LlamaIndex with our hardcoded settings for OpenRouter LLM and local embeddings.


## 🔄 Multimodal vs. Unimodal Vector Index Creation

### Understanding the Key Difference

While **multimodal** and **unimodal** RAG systems use the same underlying `VectorStoreIndex.from_documents()` method, there's a **critical difference** in how existing indexes are loaded that affects system behavior and reliability.

### 📊 Index Creation Comparison

| Aspect | Unimodal (Academic Papers) | Multimodal (This Notebook) | Impact |
|--------|----------------------------|----------------------------|---------|
| **Document Types** | Single type (PDF papers) | Multiple types (PDF, CSV, HTML, Images, Audio) | Different processing pipelines |
| **Index Creation** | `VectorStoreIndex.from_documents()` | `VectorStoreIndex.from_documents()` | **Identical** |
| **Storage Context** | ✅ Full StorageContext persistence | ✅ Full StorageContext persistence | **Identical** |
| **Index Loading** | `load_index_from_storage()` | `VectorStoreIndex.from_vector_store()` | **⚠️ Different!** |
| **Metadata Complexity** | Single file type metadata | Rich cross-modal metadata | More complex relationships |

### 🔍 The Critical Loading Difference

**Unimodal Loading (Academic Papers):**
```python
# ROBUST: Complete index reconstruction
storage_context = StorageContext.from_defaults(
    persist_dir=str(index_path), 
    vector_store=vector_store
)
index = load_index_from_storage(storage_context)
# ✅ Perfect restoration with all metadata and relationships
```

**Multimodal Loading (This Notebook):**
```python
# BASIC: Vector-only reconstruction
storage_context = StorageContext.from_defaults(
    persist_dir=str(index_path), 
    vector_store=vector_store
)
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    storage_context=storage_context
)
# ⚠️ May lose some complex relationships between file types
```

### 🎯 Why This Difference Matters

**For Unimodal Systems:**
- Documents are homogeneous (all PDFs)
- `load_index_from_storage()` ensures perfect reconstruction
- Critical for academic reproducibility

**For Multimodal Systems:**
- Documents are heterogeneous (PDFs, images, audio, CSV)
- `from_vector_store()` focuses on vector similarity
- Cross-modal relationships handled differently
- May prioritize performance over perfect metadata preservation

### 📈 Practical Implications

| Scenario | Unimodal Advantage | Multimodal Trade-off |
|----------|-------------------|---------------------|
| **Research Reproducibility** | 🎯 Identical results every time | ⚠️ Minor variations possible |
| **Cross-Modal Queries** | ❌ Not applicable | ✅ Query across file types |
| **System Startup** | ⚡ Fastest (complete restoration) | 🔄 Fast (vector-based loading) |
| **Metadata Fidelity** | 🔒 100% preserved | 📊 Core metadata preserved |
| **File Type Diversity** | 📄 Single type (PDFs) | 🌈 Multiple types supported |

### 🛠️ When to Use Each Approach

**Choose Unimodal (`load_index_from_storage`) When:**
- Working with homogeneous document types
- Perfect reproducibility is critical
- Academic research or compliance requirements
- Complex document relationships matter

**Choose Multimodal (`from_vector_store`) When:**
- Processing diverse file types simultaneously
- Cross-modal search is the priority
- Performance over perfect metadata preservation
- Building versatile content search systems

### 🎨 The Multimodal Advantage

Despite the loading difference, multimodal indexing provides unique capabilities:

1. **🔍 Cross-Modal Search**: Find information across PDFs, images, and data files
2. **📊 Rich Content Types**: Handle structured (CSV) and unstructured (text) data together  
3. **🎵 Audio Integration**: Include transcribed audio content in searches
4. **🖼️ Visual Content**: Extract information from charts and diagrams
5. **📈 Unified Knowledge Base**: Single search across all organizational content

### 💡 Best Practice Recommendation

For production multimodal systems, consider implementing **hybrid loading**:

```python
# Try robust loading first, fallback to vector-only
try:
    index = load_index_from_storage(storage_context)  # Full restoration
    print("✓ Complete index restoration")
except:
    index = VectorStoreIndex.from_vector_store(vector_store)  # Vector fallback
    print("✓ Vector-based index loading")
```

This gives you the reliability of unimodal loading with the flexibility of multimodal processing.


In [6]:
# LlamaIndex configuration with hardcoded settings
from llama_index.core import Settings
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.node_parser import SentenceSplitter

def configure_llamaindex_settings():
    """Configure LlamaIndex global settings using hardcoded configuration."""
    
    # Set up LLM with OpenRouter using hardcoded model
    Settings.llm = OpenRouter(
        api_key=os.getenv("OPENROUTER_API_KEY"),
        model=CONFIG["llm_model"]
    )
    print(f"✓ LLM configured: {CONFIG['llm_model']}")

    # Set up local embedding model (downloads locally first time, then cached)
    Settings.embed_model = resolve_embed_model(CONFIG["embedding_model"])
    print(f"✓ Embedding model configured: {CONFIG['embedding_model']}")

    # Set up node parser for chunking with hardcoded settings
    Settings.node_parser = SentenceSplitter(
        chunk_size=CONFIG["chunk_size"], 
        chunk_overlap=CONFIG["chunk_overlap"]
    )
    print(f"✓ Text chunking configured: {CONFIG['chunk_size']} chars with {CONFIG['chunk_overlap']} overlap")

# Configure the settings
configure_llamaindex_settings()
print("✓ LlamaIndex settings configured for multimodal processing")


✓ LLM configured: gpt-5-mini
✓ Embedding model configured: local:BAAI/bge-small-en-v1.5
✓ Text chunking configured: 1024 chars with 100 overlap
✓ LlamaIndex settings configured for multimodal processing


## 3. Exploring Our Multimodal Dataset

Let's examine the different types of files we have available for processing. This will show the diversity of data types that SimpleDirectoryReader can handle.


In [7]:
def explore_dataset(data_path: str = None):
    """
    Explore and categorize the files in our dataset by type.
    
    Args:
        data_path (str): Path to the data directory
    """
    if data_path is None:
        data_path = CONFIG["data_path"]
        
    data_dir = Path(data_path)
    if not data_dir.exists():
        print(f"Data directory not found: {data_dir}")
        return
    
    # Categorize files by type
    file_types = {}
    all_files = []
    
    # Walk through all files recursively
    for file_path in data_dir.rglob("*"):
        if file_path.is_file():
            suffix = file_path.suffix.lower()
            file_size = file_path.stat().st_size
            
            if suffix not in file_types:
                file_types[suffix] = []
            
            file_info = {
                "path": str(file_path),
                "name": file_path.name,
                "size_mb": round(file_size / (1024 * 1024), 2),
                "size_bytes": file_size
            }
            
            file_types[suffix].append(file_info)
            all_files.append(file_info)
    
    # Display summary
    print("🗂️  Dataset Overview")
    print("=" * 50)
    print(f"Total files found: {len(all_files)}")
    
    print(f"\n📁 File Types Distribution:")
    for file_type, files in sorted(file_types.items()):
        if file_type:  # Skip files without extension
            total_size = sum(f["size_mb"] for f in files)
            print(f"  {file_type}: {len(files)} files ({total_size:.2f} MB)")
            
            # Show file details
            for file_info in files[:3]:  # Show first 3 files of each type
                print(f"    - {file_info['name']} ({file_info['size_mb']} MB)")
            if len(files) > 3:
                print(f"    ... and {len(files) - 3} more")
    
            print()
    
    return file_types, all_files

# Explore our dataset
file_types, all_files = explore_dataset()
print(f"✓ Found {len(all_files)} files across {len(file_types)} different file types")


🗂️  Dataset Overview
Total files found: 21

📁 File Types Distribution:
  .csv: 4 files (0.00 MB)
    - italian_recipes.csv (0.0 MB)
    - agent_performance_benchmark.csv (0.0 MB)
    - agent_evaluation_metrics.csv (0.0 MB)
    ... and 1 more

  .html: 2 files (0.00 MB)
    - fitness_tracker.html (0.0 MB)
    - agent_tutorial.html (0.0 MB)

  .md: 4 files (0.00 MB)
    - recipe_instructions.md (0.0 MB)
    - agent_framework_comparison.md (0.0 MB)
    - market_analysis.md (0.0 MB)
    ... and 1 more

  .mp3: 3 files (2.95 MB)
    - rags.mp3 (0.81 MB)
    - ai_agents.mp3 (1.54 MB)
    - in_the_end.mp3 (0.6 MB)

  .pdf: 2 files (1.92 MB)
    - AI_Agent_Frameworks.pdf (0.34 MB)
    - Emerging_Agent_Architectures.pdf (1.58 MB)

  .png: 6 files (0.55 MB)
    - recipe_popularity.png (0.04 MB)
    - agent_types_comparison.png (0.1 MB)
    - agent_performance_comparison.png (0.17 MB)
    ... and 3 more

✓ Found 21 files across 6 different file types


## 4. Basic Multimodal Document Loading

Now let's use SimpleDirectoryReader to load all files from our data directory. This demonstrates the core multimodal capability.


### 🔍 Index Creation Implementation Note

The following implementation uses **multimodal-optimized loading** - notice the difference from the academic papers notebook:

#### Key Implementation Differences:

1. **Index Loading Method**: Uses `VectorStoreIndex.from_vector_store()` instead of `load_index_from_storage()`
2. **Reasoning**: Optimized for cross-modal search performance over perfect metadata preservation  
3. **Trade-off**: Slightly less metadata fidelity but better handling of diverse file types
4. **Benefit**: More flexible loading for heterogeneous document collections

This approach prioritizes the core multimodal capability while maintaining good performance and reliability.


In [8]:
from llama_index.core import SimpleDirectoryReader

def load_multimodal_documents(data_path: str = None, recursive: bool = True):
    """
    Load documents from multiple file types using SimpleDirectoryReader.
    
    Args:
        data_path (str): Path to directory containing multimodal data
        recursive (bool): Whether to search subdirectories
        
    Returns:
        List of Document objects
    """
    if data_path is None:
        data_path = CONFIG["data_path"]
        
    print(f"📂 Loading multimodal documents from: {data_path}")
    
    # Create SimpleDirectoryReader with recursive search
    reader = SimpleDirectoryReader(
        input_dir=data_path,
        recursive=recursive,
        # Let SimpleDirectoryReader handle all supported file types automatically
    )
    
    print("🔄 Processing files...")
    start_time = time.time()
    
    # Load all documents
    documents = reader.load_data()
    
    end_time = time.time()
    
    print(f"✅ Successfully loaded {len(documents)} documents in {end_time - start_time:.2f} seconds")
    
    # Analyze loaded documents by file type
    doc_types = {}
    for doc in documents:
        file_type = doc.metadata.get('file_type', 'unknown')
        if file_type not in doc_types:
            doc_types[file_type] = []
        doc_types[file_type].append(doc)
    
    print(f"\n📊 Documents by MIME type:")
    for mime_type, docs in sorted(doc_types.items()):
        print(f"  {mime_type}: {len(docs)} documents")
    
    return documents

# Load all multimodal documents
documents = load_multimodal_documents()

# Show sample document information
if documents:
    print(f"\n📄 Sample Document Analysis:")
    sample_doc = documents[0]
    print(f"File: {sample_doc.metadata.get('file_name', 'Unknown')}")
    print(f"Type: {sample_doc.metadata.get('file_type', 'Unknown')}")
    print(f"Size: {sample_doc.metadata.get('file_size', 0)} bytes")
    print(f"Text preview: {sample_doc.text[:200]}...")
    print(f"Metadata keys: {list(sample_doc.metadata.keys())}")


📂 Loading multimodal documents from: ../data
🔄 Processing files...


/Users/ishandutta/miniconda3/envs/accelerator/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/ishandutta/miniconda3/envs/accelerator/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/ishandutta/miniconda3/envs/accelerator/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Successfully loaded 42 documents in 14.85 seconds

📊 Documents by MIME type:
  application/pdf: 23 documents
  audio/mpeg: 3 documents
  image/png: 6 documents
  text/csv: 4 documents
  text/html: 2 documents
  unknown: 4 documents

📄 Sample Document Analysis:
File: AI_Agent_Frameworks.pdf
Type: application/pdf
Size: 360523 bytes
Text preview: A Comprehensive Survey of AI Agent Frameworks
and Their Applications in Financial Services
Satyadhar Joshi
Independent
Alumnus, International MBA, Bar-Ilan University, Israel
satyadhar.joshi@gmail.com...
Metadata keys: ['page_label', 'file_name', 'file_path', 'file_type', 'file_size', 'creation_date', 'last_modified_date']


## 5. Creating Multimodal Vector Index

Now let's create a vector index that can handle our multimodal documents using LanceDB for efficient storage and retrieval.


In [9]:
# Vector store and index creation
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.core import StorageContext, VectorStoreIndex

def create_multimodal_vector_store(vector_db_path: str = None):
    """Create and configure LanceDB vector store for multimodal data."""
    if vector_db_path is None:
        vector_db_path = CONFIG["vector_db_path"]
        
    try:
        import lancedb
        
        # Create storage directory
        Path(vector_db_path).parent.mkdir(parents=True, exist_ok=True)
        
        # Connect to LanceDB
        db = lancedb.connect(str(vector_db_path))
        print(f"✓ Connected to LanceDB at: {vector_db_path}")
        
        # Create vector store
        vector_store = LanceDBVectorStore(
            uri=str(vector_db_path), 
            table_name="multimodal_documents"
        )
        print("✓ LanceDB vector store created for multimodal data")
        
        return vector_store
        
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return None

def create_multimodal_index(documents: List, 
                           vector_store, 
                           index_storage_path: str = None,
                           force_rebuild: bool = False):
    """Create or load a multimodal vector index."""
    
    if index_storage_path is None:
        index_storage_path = CONFIG["index_storage_path"]
    
    index_path = Path(index_storage_path)
    index_path.mkdir(parents=True, exist_ok=True)
    
    # Check if index already exists
    index_store_file = index_path / "index_store.json"
    
    if not force_rebuild and index_store_file.exists():
        print("📁 Loading existing multimodal index...")
        try:
            storage_context = StorageContext.from_defaults(
                persist_dir=str(index_path), 
                vector_store=vector_store
            )
            
            index = VectorStoreIndex.from_vector_store(
                vector_store=vector_store,
                storage_context=storage_context
            )
            print("✓ Successfully loaded existing multimodal index")
            return index
            
        except Exception as e:
            print(f"⚠️  Error loading existing index: {e}")
            print("Creating new index...")
    
    if not documents:
        print("❌ No documents to index")
        return None
    
    print("🔨 Creating new multimodal vector index...")
    start_time = time.time()
    
    # Create storage context with vector store
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    # Create index with progress bar
    index = VectorStoreIndex.from_documents(
        documents, 
        storage_context=storage_context, 
        show_progress=True
    )
    
    end_time = time.time()
    print(f"✓ Multimodal index created in {end_time - start_time:.2f} seconds")
    
    # Save index to storage
    print("💾 Saving multimodal index to storage...")
    index.storage_context.persist(persist_dir=str(index_path))
    print("✓ Index saved successfully")
    
    return index

# Create vector store and index for multimodal data
print("🚀 Setting up multimodal vector storage...")
multimodal_vector_store = create_multimodal_vector_store()

if multimodal_vector_store and documents:
    multimodal_index = create_multimodal_index(
        documents=documents, 
        vector_store=multimodal_vector_store,
        force_rebuild=False
    )
    
    if multimodal_index:
        print("✅ Multimodal RAG system ready for cross-modal queries!")
    else:
        print("❌ Failed to create multimodal index")
else:
    print("❌ Vector store creation failed or no documents available")


2025-09-20 12:57:10,572 - WARNING - Table multimodal_documents doesn't exist yet. Please add some data to create it.


🚀 Setting up multimodal vector storage...
✓ Connected to LanceDB at: storage/multimodal_vectordb
✓ LanceDB vector store created for multimodal data
🔨 Creating new multimodal vector index...


Generating embeddings: 100%|██████████| 55/55 [00:02<00:00, 21.76it/s]
2025-09-20 12:57:13,598 - INFO - Create new table multimodal_documents adding data.


✓ Multimodal index created in 3.07 seconds
💾 Saving multimodal index to storage...
✓ Index saved successfully
✅ Multimodal RAG system ready for cross-modal queries!


[2025-09-20T07:27:13Z WARN  lance::dataset::write::insert] No existing dataset at /Users/ishandutta/Documents/code/ai-accelerator/Day_6/session_2/llamaindex_rag/storage/multimodal_vectordb/multimodal_documents.lance, it will be created


## 6. Multimodal Query Engine and Cross-Modal Search

Now let's create a query engine that can search across all our different data types and demonstrate cross-modal queries.


In [10]:
# Query engine setup
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever

def setup_multimodal_query_engine(index, similarity_top_k: int = None):
    """Setup query engine for multimodal search."""
    if similarity_top_k is None:
        similarity_top_k = CONFIG["similarity_top_k"]
        
    if not index:
        print("❌ Index not available. Please create index first.")
        return None
    
    try:
        # Create retriever for multimodal search
        retriever = VectorIndexRetriever(
            index=index,
            similarity_top_k=similarity_top_k,
        )
        print(f"✓ Multimodal retriever configured to find top {similarity_top_k} similar chunks")
        
        # Create query engine
        query_engine = RetrieverQueryEngine(retriever=retriever)
        print("✓ Multimodal query engine setup successfully")
        
        return query_engine
        
    except Exception as e:
        print(f"❌ Error setting up query engine: {e}")
        return None

def search_multimodal_documents(query_engine, query: str, include_metadata: bool = True) -> Dict[str, any]:
    """Search across multimodal documents and return detailed results."""
    if not query_engine:
        return {
            "success": False,
            "error": "Query engine not initialized.",
            "response": "",
            "sources": [],
        }
    
    try:
        print(f"🔍 Searching across multimodal data: '{query}'")
        start_time = time.time()
        
        # Query the multimodal RAG system
        response = query_engine.query(query)
        
        end_time = time.time()
        
        # Extract source information from retrieved nodes
        sources = []
        if hasattr(response, "source_nodes"):
            for node in response.source_nodes:
                source_info = {
                    "text": (
                        node.text[:300] + "..."
                        if len(node.text) > 300
                        else node.text
                    ),
                    "score": getattr(node, "score", 0.0),
                }
                
                # Add metadata if available and requested
                if include_metadata and hasattr(node, "metadata"):
                    metadata = node.metadata
                    source_info.update({
                        "file_name": metadata.get("file_name", "Unknown"),
                        "file_type": metadata.get("file_type", "Unknown"),
                        "file_path": metadata.get("file_path", "Unknown"),
                        "file_size": metadata.get("file_size", 0),
                    })
                
                sources.append(source_info)
        
        result = {
            "success": True,
            "response": str(response),
            "sources": sources,
            "query": query,
            "search_time": end_time - start_time,
            "num_sources": len(sources),
        }
        
        print(f"✓ Search completed in {end_time - start_time:.2f} seconds")
        print(f"📚 Found {len(sources)} relevant sources across different file types")
        
        return result
        
    except Exception as e:
        print(f"❌ Error during search: {e}")
        return {"success": False, "error": str(e), "response": "", "sources": []}

# Setup multimodal query engine
if 'multimodal_index' in locals() and multimodal_index:
    multimodal_query_engine = setup_multimodal_query_engine(multimodal_index)
    
    if multimodal_query_engine:
        print("🚀 Multimodal query engine ready for cross-modal search!")
    else:
        print("❌ Failed to setup multimodal query engine")
else:
    print("❌ Multimodal index not available")


✓ Multimodal retriever configured to find top 5 similar chunks
✓ Multimodal query engine setup successfully
🚀 Multimodal query engine ready for cross-modal search!


## 7. Interactive Multimodal Query Examples

Let's demonstrate the power of our multimodal RAG system with cross-modal queries that search across different data types simultaneously.


In [11]:
def ask_multimodal_question(query_engine, question: str, show_sources: bool = True):
    """
    Ask a custom question to the multimodal RAG system and display results.
    
    Args:
        query_engine: The configured multimodal query engine
        question (str): Your question about the multimodal data
        show_sources (bool): Whether to display source information
    """
    print(f"❓ Multimodal Question: {question}")
    print("=" * 70)
    
    result = search_multimodal_documents(query_engine, question, include_metadata=True)
    
    if result["success"]:
        print(f"💡 Answer:")
        print(result["response"])
        print(f"\n📊 Search completed in {result['search_time']:.2f} seconds")
        print(f"📚 Found {result['num_sources']} relevant sources across different data types")
        
        if show_sources and result["sources"]:
            # Show file type distribution
            file_types = {}
            for source in result["sources"]:
                file_type = source.get("file_type", "unknown")
                if file_type not in file_types:
                    file_types[file_type] = 0
                file_types[file_type] += 1
            
            print(f"\n📁 Source File Types: {dict(file_types)}")
            
            print(f"\n📖 Top Sources:")
            for i, source in enumerate(result["sources"][:3], 1):
                print(f"{i}. {source.get('file_name', 'Unknown')} ({source.get('file_type', 'Unknown')})")
                print(f"   Score: {source.get('score', 0):.3f}")
                print(f"   Content: {source['text'][:150]}...")
                print()
                
    else:
        print(f"❌ Error: {result['error']}")

# # Example multimodal queries
# multimodal_queries = [
#     # "What are the performance benchmarks for different AI agents?",
#     # "How do I configure a ReAct agent for research tasks?", 
#     # "What are the architectural patterns discussed in the agent frameworks?",
#     # "Which AI agent has the best accuracy score?",
#     # "What are the cost implications of different agent models?"
#     "What is the accuracy_score for the ReAct agent?"
# ]

# print("🎯 Multimodal Query Demonstrations")
# print("=" * 60)

# # Run a few example queries
# for i, question in enumerate(multimodal_queries[:3], 1):
#     print(f"\n{'='*20} Example {i} {'='*20}")
    
#     if 'multimodal_query_engine' in locals() and multimodal_query_engine:
#         ask_multimodal_question(multimodal_query_engine, question, show_sources=True)
#     else:
#         print("❌ Multimodal query engine not available")
    
#     if i < 3:
#         print("\n" + "="*60)

# Demo queries for diverse data types
diverse_queries = [
    "What is the prep time for Spaghetti Carbonara?",  # Should hit cooking CSV
    "Which stock had the highest return in my portfolio?",  # Should hit finance CSV
    "What is the best time to visit Tokyo?",  # Should hit travel markdown
    "How many calories did I burn on Tuesday?",  # Should hit health HTML
    "What are the steps to make Carbonara?",  # Should hit cooking markdown
    "What was NVIDIA's performance?",  # Should hit finance data
]

print("🎯 Testing Diverse Multimodal Queries")
print("=" * 60)

# Test one query from each topic
for i, question in enumerate(diverse_queries[:3], 1):
    print(f"\n{'='*15} Query {i}: {question[:30]}... {'='*15}")
    ask_multimodal_question(multimodal_query_engine, question, show_sources=True)
    print("\n" + "="*70)

# Custom question area
print(f"\n{'='*20} Custom Question {'='*20}")
custom_question = "What is the prep time for Italian recipes?"
ask_multimodal_question(multimodal_query_engine, custom_question, show_sources=True)


🎯 Testing Diverse Multimodal Queries

=============== Query 1: What is the prep time for Spag... ===============
❓ Multimodal Question: What is the prep time for Spaghetti Carbonara?
🔍 Searching across multimodal data: 'What is the prep time for Spaghetti Carbonara?'


2025-09-20 12:57:24,186 - INFO - query_type :, vector
2025-09-20 12:57:27,616 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-20 12:57:30,526 - INFO - query_type :, vector


✓ Search completed in 6.71 seconds
📚 Found 5 relevant sources across different file types
💡 Answer:
15 minutes

📊 Search completed in 6.71 seconds
📚 Found 5 relevant sources across different data types

📁 Source File Types: {'Unknown': 1, 'text/csv': 2, 'image/png': 1, 'application/pdf': 1}

📖 Top Sources:
1. recipe_instructions.md (Unknown)
   Score: 0.675
   Content: # 🍝 Classic Spaghetti Carbonara Recipe

## Ingredients
- 400g spaghetti pasta
- 4 large egg yolks
- 100g pecorino romano cheese (grated)
- 150g guanci...

2. italian_recipes.csv (text/csv)
   Score: 0.505
   Content: Spaghetti Carbonara, Italian, 20, Easy, Pasta, 450
Margherita Pizza, Italian, 45, Medium, Tomato, 320
Risotto Milanese, Italian, 35, Hard, Rice, 380
T...

3. agent_performance_benchmark.csv (text/csv)
   Score: 0.400
   Content: ReAct-GPT4, reasoning, 0.87, 1200, 45.2, 0.02, langchain
AutoGPT, autonomous, 0.78, 2100, 78.5, 0.035, autogpt
LangChain-Agent, tool_using, 0.82, 950,...



=============== Query 2: 

2025-09-20 12:57:31,779 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-20 12:57:36,000 - INFO - query_type :, vector


✓ Search completed in 5.51 seconds
📚 Found 5 relevant sources across different file types
💡 Answer:
Empty Response

📊 Search completed in 5.51 seconds
📚 Found 5 relevant sources across different data types

📁 Source File Types: {'text/csv': 1, 'image/png': 1, 'Unknown': 1, 'application/pdf': 2}

📖 Top Sources:
1. investment_portfolio.csv (text/csv)
   Score: 0.552
   Content: Stock, AAPL, Apple Inc, 10000, 12500, 25.0, Medium
Stock, GOOGL, Alphabet Inc, 8000, 9200, 15.0, Medium
Stock, TSLA, Tesla Inc, 5000, 4200, -16.0, Hig...

2. stock_performance.png (image/png)
   Score: 0.472
   Content: ...

3. market_analysis.md (Unknown)
   Score: 0.449
   Content: # 📈 Q3 2024 Market Analysis Report

## Executive Summary

The third quarter of 2024 showed mixed performance across different asset classes, with tech...



=============== Query 3: What is the best time to visit... ===============
❓ Multimodal Question: What is the best time to visit Tokyo?
🔍 Searching across multimodal data: 'What i

2025-09-20 12:57:37,280 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-20 12:57:43,231 - INFO - query_type :, vector


✓ Search completed in 7.23 seconds
📚 Found 5 relevant sources across different file types
💡 Answer:
The best times to visit Tokyo are March–May (cherry blossom season) and September–November.

📊 Search completed in 7.23 seconds
📚 Found 5 relevant sources across different data types

📁 Source File Types: {'Unknown': 2, 'image/png': 1, 'application/pdf': 2}

📖 Top Sources:
1. city_guides.md (Unknown)
   Score: 0.545
   Content: # Ultimate City Travel Guide

## Paris, France 🇫🇷

**Best Time to Visit:** April-June, September-October
**Must-See Attractions:**
- Eiffel Tower - Ic...

2. recipe_instructions.md (Unknown)
   Score: 0.344
   Content: # 🍝 Classic Spaghetti Carbonara Recipe

## Ingredients
- 400g spaghetti pasta
- 4 large egg yolks
- 100g pecorino romano cheese (grated)
- 150g guanci...

3. city_temperatures.png (image/png)
   Score: 0.337
   Content: ...



==================== Custom Question ====================
❓ Multimodal Question: What is the prep time for Italian recipes?


2025-09-20 12:57:44,554 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


✓ Search completed in 21.89 seconds
📚 Found 5 relevant sources across different file types
💡 Answer:
Empty Response

📊 Search completed in 21.89 seconds
📚 Found 5 relevant sources across different data types

📁 Source File Types: {'Unknown': 2, 'text/csv': 1, 'image/png': 1, 'audio/mpeg': 1}

📖 Top Sources:
1. recipe_instructions.md (Unknown)
   Score: 0.572
   Content: # 🍝 Classic Spaghetti Carbonara Recipe

## Ingredients
- 400g spaghetti pasta
- 4 large egg yolks
- 100g pecorino romano cheese (grated)
- 150g guanci...

2. italian_recipes.csv (text/csv)
   Score: 0.547
   Content: Spaghetti Carbonara, Italian, 20, Easy, Pasta, 450
Margherita Pizza, Italian, 45, Medium, Tomato, 320
Risotto Milanese, Italian, 35, Hard, Rice, 380
T...

3. city_guides.md (Unknown)
   Score: 0.405
   Content: # Ultimate City Travel Guide

## Paris, France 🇫🇷

**Best Time to Visit:** April-June, September-October
**Must-See Attractions:**
- Eiffel Tower - Ic...



In [12]:
custom_question="in the end it doesn't even matter"
ask_multimodal_question(multimodal_query_engine, custom_question, show_sources=True)

❓ Multimodal Question: in the end it doesn't even matter
🔍 Searching across multimodal data: 'in the end it doesn't even matter'


2025-09-20 12:58:16,704 - INFO - query_type :, vector
2025-09-20 12:58:18,223 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


✓ Search completed in 7.48 seconds
📚 Found 5 relevant sources across different file types
💡 Answer:
Empty Response

📊 Search completed in 7.48 seconds
📚 Found 5 relevant sources across different data types

📁 Source File Types: {'audio/mpeg': 2, 'application/pdf': 2, 'text/csv': 1}

📖 Top Sources:
1. in_the_end.mp3 (audio/mpeg)
   Score: 0.433
   Content: I tried so hard and got so far In the end, it doesn't even matter I had to fall to lose it all In the end, it doesn't even matter...

2. Emerging_Agent_Architectures.pdf (application/pdf)
   Score: 0.375
   Content: Message subscribing or filtering improves multi-agent
performance by ensuring agents only receive information relevant to their tasks.
In vertical arc...

3. Emerging_Agent_Architectures.pdf (application/pdf)
   Score: 0.368
   Content: complete problems [16, 23, 32]. They often do this by breaking a larger problem into smaller subproblems, and then
solving each one with the appropria...



## Conclusion

🎉 **Congratulations!** You have successfully built an advanced **Multimodal RAG System** using LlamaIndex's `SimpleDirectoryReader` with comprehensive cross-modal capabilities.

### What We Accomplished

This tutorial demonstrated building a RAG system that can handle multiple data types:

#### 1. **Multimodal Document Loading**
- ✅ **PDF Documents**: Academic research papers on AI agents
- ✅ **CSV Files**: Agent performance benchmarks and evaluation metrics  
- ✅ **Markdown Files**: Framework comparisons and documentation
- ✅ **HTML Files**: Tutorial and instructional content
- ✅ **Image Files**: Charts, diagrams, and visual content
- ✅ **Audio Files**: Supplementary audio content

#### 2. **Key Features Implemented**
- ✅ **Hardcoded Configuration**: No external config files needed
- ✅ **Cross-Modal Search**: Query across all file types simultaneously
- ✅ **Semantic Similarity**: Find relevant content regardless of source format
- ✅ **Source Attribution**: Track which file types contributed to answers
- ✅ **LanceDB Vector Store**: Efficient multimodal document storage
- ✅ **OpenRouter Integration**: Using `gpt-4o` for response generation
- ✅ **Local Embeddings**: `BAAI/bge-small-en-v1.5` for cost-effective embedding

#### 3. **SimpleDirectoryReader Capabilities**
According to the [official documentation](https://developers.llamaindex.ai/python/framework/module_guides/loading/simpledirectoryreader/), we successfully utilized:

```python
# Basic multimodal loading
SimpleDirectoryReader(input_dir="../../data", recursive=True)

# Advanced features available
SimpleDirectoryReader(
    input_dir="path/to/directory",
    recursive=True,                    # Search subdirectories
    required_exts=[".pdf", ".csv"],    # Filter file types
    exclude=["file1.txt"],            # Exclude specific files
    file_metadata=custom_func,         # Custom metadata extraction
    num_files_limit=100,              # Limit number of files
    encoding="utf-8"                  # Specify encoding
)
```

### Real-World Applications

This multimodal RAG system can be applied to:

- **Research and Academia**: Query across papers, datasets, and supplementary materials
- **Documentation Systems**: Search technical docs, tutorials, configs, and examples
- **Business Intelligence**: Combine reports, spreadsheets, presentations, and recordings
- **Content Management**: Organize and search diverse content libraries
- **Knowledge Bases**: Build comprehensive Q&A systems with diverse source materials

### Next Steps

1. **Extend File Types**: Add `.docx`, `.pptx`, `.epub` support
2. **Custom Metadata**: Implement domain-specific metadata extraction
3. **Hybrid Search**: Combine vector search with keyword search
4. **Performance Optimization**: Use iterative loading for large datasets
5. **Multi-Language Support**: Test with international documents

### Usage Tips

- **Query Optimization**: Use specific queries that benefit from cross-modal information
- **File Organization**: Structure data directories logically
- **Custom Questions**: Modify the `custom_question` variable to test your own queries
- **Monitor Sources**: Check file type distribution in results to understand retrieval patterns

Happy building with multimodal RAG! 🚀📚🔍

---

**Ready to explore?** Run the cells above and try your own questions with the interactive query interface!
